**Define what we want to plot**

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings(category=UserWarning, action="ignore", module='dacbench')
warnings.filterwarnings(category=FutureWarning, action="ignore", module='dacbench')

project_name = 'CANDID_DAC'
config_id = 'best'
metric_to_plot = "avg_reward_test_set"

# benchmarks_to_compare = ['sigmoid', 'candid_sigmoid', 'piecewise_linear']
benchmarks_to_compare = ['sigmoid', 'piecewise_linear']
dim = 5
# for piecewise_linear or candid_sigmoid we will have to specify action importances and parameters of reward function
reward_shape = 'exponential'
exp_reward= 4.6
importance_base = 0.5
n_act = 3

path_to_config = f'../run_data/{project_name}_configs.csv'

df_config = pd.read_csv(path_to_config)

**Get the runs per benchmark we want to plot**

In [ ]:
from plotting_helpers import get_run_configs

runs_per_benchmark = {}

for benchmark in benchmarks_to_compare:
    run_configs_on_bench = get_run_configs(df_config, dim=dim, benchmark=benchmark, config_id=config_id,
                                           importance_base=importance_base, reward_shape=reward_shape, exp_reward=exp_reward,
                                           n_act=n_act)
    seeds_per_algorithm = run_configs_on_bench.groupby('algorithm')['seed'].aggregate(['count', list])
    seeds_per_algorithm['list'] = seeds_per_algorithm['list'].apply(lambda x: sorted(x))

    # remove pandas max column width, and also increase the width of the table
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 1000)
    print(f'For benchmark {benchmark} we have the following runs:')
    print(seeds_per_algorithm)
    runs_per_benchmark[benchmark] = run_configs_on_bench.groupby('algorithm')['run_id'].aggregate(list)
    run_configs_on_bench.index

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plotting_helpers import METHOD_COLOURS, get_run_configs, translate_run_name, get_best_possible_avg_reward

metrics_path = f'../run_data/{project_name}_metrics.csv'
df_metrics = pd.read_csv(metrics_path)

plot_sig_on_train = True  # if True and plotting is usually done on test set, additionally train set performances on sigmoid

# width = 4
# fig, axes = plt.subplots(1, len(benchmarks_to_compare), figsize=(len(benchmarks_to_compare) * width, width), sharey=True, sharex=True)
plt.rcParams.update({
    'font.size': 8,           # Global font size
    'axes.titlesize': 7,      # Title size of individual plots
    'axes.labelsize': 7,      # Label size for x and y labels
    'xtick.labelsize': 6.5,      # Size of x-tick labels
    'ytick.labelsize': 6.5,      # Size of y-tick labels
    'legend.fontsize': 7,      # Size of the legend text
    'figure.titlesize': 12,     # Title size of the entire figure
    'lines.linewidth': 0.75
})
height = 1.2
width = 3.5*height
dpi=100  # set to 100 to get in size of paper not zoomed in
fig, axes = plt.subplots(1, len(benchmarks_to_compare), figsize=(width, height), sharey=True, dpi=dpi, )

for i, benchmark in enumerate(benchmarks_to_compare):
    ax = axes[i]
    configs = runs_per_benchmark[benchmark]

    if benchmark in ['candid_sigmoid', 'piecewise_linear']:
        optim_1D = get_best_possible_avg_reward(5, benchmark=benchmark, reward_shape=reward_shape, c=exp_reward, importance_base=importance_base, max_dim=1)
        optim_5D = get_best_possible_avg_reward(5, benchmark=benchmark, reward_shape=reward_shape, c=exp_reward, importance_base=importance_base, max_dim=5)

        ax.plot([0, 10000], [optim_5D, optim_5D], color='black', linestyle='--', label='optimal')
        ax.plot([0, 10000], [optim_1D, optim_1D], color='dimgray', linestyle='--', label='optimal (1D)')
        if benchmark == 'piecewise_linear':
            ax.set_title(f'{dim}D Piecewise Linear', y=0.94)
        else:
            ax.set_title(f'{dim}D CANDID Sigmoid', y=0.94)
    else:
        optim_5D = get_best_possible_avg_reward(5, benchmark=benchmark, reward_shape=reward_shape, c=exp_reward, importance_base=importance_base, max_dim=5)
        ax.plot([0, 10000], [optim_5D, optim_5D], color='black', linestyle='--', label='Accumulated optimal')
        ax.set_title(f'{dim}D Sigmoid', y=0.94)

    for run_name, run_ids in configs.items():
        df_rewards = df_metrics[df_metrics['run_id'].isin(run_ids)]

        run_name = run_name.rsplit('_', 1)[0]
        label = translate_run_name(run_name)

        # remove all rows containing NaN values for the metric to plot
        if benchmark == 'sigmoid' and plot_sig_on_train:
            df_train = df_metrics[df_metrics['run_id'].isin(run_ids)]
            df_train = df_train.dropna(subset=['avg_reward_train_set'])
            df_train = df_train.groupby('_step')['avg_reward_train_set'].agg('mean')
            ax.plot(df_train.index / 10, df_train, linestyle=':', color=METHOD_COLOURS[run_name])

        df_rewards = df_rewards.dropna(subset=[metric_to_plot]) 
        df_rewards = df_rewards.groupby('_step')[metric_to_plot].agg(['mean', 'std'])


        ax.set_ylim(0, 10)
        ax.set_xlim(0, 10000)
        ax.set_yticks(np.linspace(0, 10, 5))
        ax.set_xticks(np.linspace(0, 10000, 5))
        ax.tick_params(axis='x', pad=-5)
        ax.set_xlabel('Episodes')

        # plot mean and std
        ax.plot(df_rewards.index / 10, df_rewards['mean'], label=label, color=METHOD_COLOURS[run_name])
        ax.fill_between(df_rewards.index / 10, df_rewards['mean'] - df_rewards['std'], df_rewards['mean'] + df_rewards['std'],
                        color=METHOD_COLOURS[run_name], alpha=0.2)
        

y_label = 'Avg. episodic reward'
axes[0].set_ylabel(y_label)

leg = axes[-1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

if plot_sig_on_train:
    from matplotlib.lines import Line2D
    handles, labels = axes[-1].get_legend_handles_labels()
    handles += [Line2D([0], [0], linestyle=':', color='gray')]
    labels += ['on train set']
    axes[-1].legend(handles=handles, labels=labels, loc='center left', bbox_to_anchor=(1, 0.6))
fig.subplots_adjust(wspace=0.15)
fig.savefig(f'paper_plots/sig_vs_pl.png', bbox_inches='tight', dpi=600)